This iteration of the API call program uses an artist list formatted as a csv as the input to do the API calls for album information.

In [6]:
import requests
import pandas as pd
import time

In [7]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': '9cdef0bc741e471385b100af843c28c5',
    'client_secret': 'a0820dd07a3e4d8facb51bb5ed5130a2',
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [8]:
def get_albums(album_df):
    a = 0
    artist_id_list = csv

    while a < len(artist_id_list):
        if a < len(artist_id_list):
            print(artist_id_list.iloc[a,0])
            artist_id = artist_id_list.iloc[a,0]
            # pull all artist's albums
            r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums', 
                            headers=headers, 
                            params={'include_groups': 'album'})
            d = r.json()
            
            time.sleep(1)
            
            for album in d['items']:
                if album['type'] == 'album':
                    band_id = album['artists'][0]['id']
                    band_name = album['artists'][0]['name']
                    album_name = album['name']
                    release_date = album['release_date']
                    total_tracks = album['total_tracks']
                    album_id = album['id']

                    # skip album doubles
                    trim_name = album_name.split('(')[0].strip()
                    # Trivium specific
                    if trim_name.upper() in albums or album['name'] == ('Ascendancy (Commentary Version)') or album['name'] == ('Ember To Inferno'):
                        continue
                    albums.append(trim_name.upper())
                    
                    #save data in pd dataframe
                    album_df = album_df.append({'band_id':band_id,'band_name':band_name,'album_name':album_name,'release_date':release_date,
                                    'total_tracks':total_tracks,'album_id':album_id},ignore_index=True)
        else:
            break 
        a += 1
    return album_df

In [9]:
#Main
BASE_URL = 'https://api.spotify.com/v1/'
csv = pd.read_csv('artist_list.csv',index_col=False)
data = []
albums = []
album_df = pd.DataFrame(columns=["band_id","band_name","album_name","release_date",
                  "total_tracks","album_id"])
album_df = get_albums(album_df)

#album_df = album_df.set_index('album_id')
album_df.to_csv(r'C:\Users\dsgra\OneDrive\Documents\01 - Full Stack Data Science PJ\data science pj\album_list.csv',encoding='utf-8-sig')
print('-----------------------------------')
print('-----------------------------------')
print('Saved to .csv in project directory.')
print('-----------------------------------')
print('-----------------------------------')

278ZYwGhdK6QTzE3MFePnP


C:\Users\dsgra\AppData\Local\Temp\ipykernel_9984\3301996681.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  album_df = album_df.append({'band_id':band_id,'band_name':band_name,'album_name':album_name,'release_date':release_date,
C:\Users\dsgra\AppData\Local\Temp\ipykernel_9984\3301996681.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  album_df = album_df.append({'band_id':band_id,'band_name':band_name,'album_name':album_name,'release_date':release_date,
C:\Users\dsgra\AppData\Local\Temp\ipykernel_9984\3301996681.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  album_df = album_df.append({'band_id':band_id,'band_name':band_name,'album_name':album_name,'release_date':release_date,
C:\Users\dsgra\AppData\Local\Te

6bu7CtcOMWcS0BMq7snHW6


C:\Users\dsgra\AppData\Local\Temp\ipykernel_9984\3301996681.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  album_df = album_df.append({'band_id':band_id,'band_name':band_name,'album_name':album_name,'release_date':release_date,
C:\Users\dsgra\AppData\Local\Temp\ipykernel_9984\3301996681.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  album_df = album_df.append({'band_id':band_id,'band_name':band_name,'album_name':album_name,'release_date':release_date,
C:\Users\dsgra\AppData\Local\Temp\ipykernel_9984\3301996681.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  album_df = album_df.append({'band_id':band_id,'band_name':band_name,'album_name':album_name,'release_date':release_date,
C:\Users\dsgra\AppData\Local\Te

65C6Unk7nhg2aCnVuAPMo8
-----------------------------------
-----------------------------------
Saved to .csv in project directory.
-----------------------------------
-----------------------------------


C:\Users\dsgra\AppData\Local\Temp\ipykernel_9984\3301996681.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  album_df = album_df.append({'band_id':band_id,'band_name':band_name,'album_name':album_name,'release_date':release_date,
C:\Users\dsgra\AppData\Local\Temp\ipykernel_9984\3301996681.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  album_df = album_df.append({'band_id':band_id,'band_name':band_name,'album_name':album_name,'release_date':release_date,
C:\Users\dsgra\AppData\Local\Temp\ipykernel_9984\3301996681.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  album_df = album_df.append({'band_id':band_id,'band_name':band_name,'album_name':album_name,'release_date':release_date,
C:\Users\dsgra\AppData\Local\Te

In [10]:
album_df

,band_id,band_name,album_name,release_date,total_tracks,album_id
0,278ZYwGhdK6QTzE3MFePnP,Trivium,In The Court Of The Dragon,2021-10-08,10,0mrtkWYrUzTuFwyiiQPdQs
1,278ZYwGhdK6QTzE3MFePnP,Trivium,What The Dead Men Say,2020-04-24,10,0aXIJYbWk4u41iJmoJmp8y
2,278ZYwGhdK6QTzE3MFePnP,Trivium,The Sin and the Sentence,2017-10-20,11,3qq44o8Qqvz5JQ7b5AevLT
3,278ZYwGhdK6QTzE3MFePnP,Trivium,Ember To Inferno: Ab Initio,2016-12-02,25,7mjOpLKh8UXgyXB1J5LvbN
4,278ZYwGhdK6QTzE3MFePnP,Trivium,Silence in the Snow (Special Edition),2015-10-02,13,5l2xUBNPkPSZB0KVrEXANI
5,278ZYwGhdK6QTzE3MFePnP,Trivium,Vengeance Falls,2013-10-11,10,5qI1swLFBDtgo9Q7lS2GcT
6,278ZYwGhdK6QTzE3MFePnP,Trivium,In Waves,2011-08-03,13,38sCkVGoLKjqFcOrTKTEC4
7,278ZYwGhdK6QTzE3MFePnP,Trivium,Shogun (Special Edition),2008-09-24,14,0kIXzVzbFuUf5kxM8US67m
8,278ZYwGhdK6QTzE3MFePnP,Trivium,The Crusade,2006-01-01,13,42MixogSAsobX5po649Y0j
9,278ZYwGhdK6QTzE3MFePnP,Trivium,Ascendancy (Special Edition),2005-03-07,16,21mOhkOtrPVySn4D2SbOAF
